# Step 7 - Exporting graphs for external visualization
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the produced graphs from 03_poi_based_generation and exports the data in a format in which it can be used in an external visualization.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-09-25  
Last modified: 2020-12-01

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Igraph to GeoJSON

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Exporting streets to GeoJSON")
    Gs = {}
    for networktype in networktypes:
        Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
        Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
    for nw, G in Gs.items():
        G_geojson = ig_to_geojson(G)
        with open(PATH["exports"] + placeid + "/" + placeid + "_" + nw + '.json', 'w') as f:
            geojson.dump(G_geojson, f)
    
    print(placeid + ": Exporting simulation results to GeoJSON")
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
        
    for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
        GT_geojson = ig_to_geojson(GT)
        with open(PATH["exports"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.json', 'w') as f:
            geojson.dump(GT_geojson, f)

## Railwaystation POIs gdf to GeoJSON

In [ ]:
poi_source_here = "railwaystation"
poi_gdf = gpd.GeoDataFrame.from_file(PATH["data"] + placeid + "/" + placeid + '_' + 'poi_' + poi_source_here + '.gpkg')
pp.pprint(poi_gdf.geometry.x)
with open(PATH["exports"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '.json', 'w') as f:
    geojson.dump(gdf_to_geojson(poi_gdf, poi_gdf.keys()), f)

## GeoJSON Linter

https://geojsonlint.com/